In [1]:
import sys
import time

import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import autokeras as ak

from sklearn.model_selection import train_test_split, KFold

sys.path.append("..")

from utils import paths
from utils.training import apply_kfold
from utils.tensorflow_keras import set_memory_growth

pd.set_option("display.max_columns", None)

tf.config.list_physical_devices()


2023-07-12 20:20:06.281532: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 20:20:06.353520: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-12 20:20:06.371648: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-12 20:20:06.757534: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
data = pd.read_csv(paths.DATA_PATH + "/COVID19MX_sample.csv")
data = data.drop(columns=["TIPO_PACIENTE", "FECHA_INGRESO", "FECHA_SINTOMAS", "OTRA_COM",
                          "OTRO_CASO", "CLASIFICACION_FINAL", "PAIS_ORIGEN", "UCI"])
data.sample(10)


,SEXO,DEFUNCION,INTUBADO,NEUMONIA,EDAD,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,PAIS_NACIONALIDAD
92312,1,1,1.0,1.0,66.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,109.0
64899,1,0,3.0,2.0,42.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,109.0
49837,1,1,2.0,1.0,85.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,109.0
40996,1,1,2.0,1.0,33.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,109.0
93442,2,1,2.0,1.0,69.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,109.0
48717,2,0,3.0,2.0,29.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,109.0
13448,1,0,3.0,2.0,23.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,109.0
56373,2,1,1.0,1.0,86.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,109.0
78703,2,0,3.0,2.0,70.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,109.0
10419,1,1,2.0,2.0,63.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,109.0


In [3]:
X = data.drop(columns=["DEFUNCION"]).to_numpy()
Y = data[["DEFUNCION"]].to_numpy()
X.shape


(93458, 15)

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=557)


In [5]:
experiments = [
    "experiment_20230711_222446",  # 15 1024 16 1
    "experiment_20230711_222532",  # 15 32 32 1
    "experiment_20230711_222628",  # 15 256 128 1
    # "experiment_20230711_222726",  # 15 32 32 1
    # "experiment_20230711_222803",  # 15 32 32 1
    # "experiment_20230711_222838",  # 15 32 32 1
    "experiment_20230711_222914",  # 15 32 64 1
    # "experiment_20230711_223003",  # 15 32 32 1
    "experiment_20230711_223049",  # 15 256 16 32 1
    # "experiment_20230711_223200",  # 15 32 32 1
]


In [6]:
set_memory_growth()
n_splits = 10

test1 = {
    "epochs": 10, "batch_size": 1024,
    "optimizer": tf.keras.optimizers.Adam(learning_rate=0.001),
    "loss": "binary_crossentropy"
}

for experiment in experiments:
    print(f"Training {experiment}")
    cnn_1_results = apply_kfold(
        experiment, test1, 
        (X_train, Y_train), (X_test, Y_test), 
        n_splits,
        load_model=True
    )
    cnn_1_results["model"] = experiment
    cnn_1_results.to_csv(paths.RESULTS_PATH + f"/{experiment}_results.csv", index=False)
    time.sleep(1)


GPU Success!
Training experiment_20230711_222446


2023-07-12 20:20:07.673682: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 20:20:07.674250: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-12 20:20:07.674380: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-12 20:20:07.674432: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA

Training experiment_20230711_222532


100%|██████████| 10/10 [00:42<00:00,  4.30s/it]


Training experiment_20230711_222628


100%|██████████| 10/10 [00:44<00:00,  4.49s/it]


Training experiment_20230711_222914


100%|██████████| 10/10 [01:00<00:00,  6.02s/it]


Training experiment_20230711_223049


100%|██████████| 10/10 [01:11<00:00,  7.12s/it]
